In [6]:
import numpy as np
import pandas as pd

# Parameters
area_width = 100
area_height = 100
sensor_count = 5
r_min = 5
r_max = 20
fixed_cost = 10

# Constants and bounds
COST_PER_RADIUS = 0.8
MIN_COVERAGE_REQ = 1
EPS = 1e-6
SWARM_SIZE = 30
MAX_ITER = 20
W_INERTIA = 0.7
C_COGNITIVE = 1.5
C_SOCIAL = 1.5
V_MAX_FACTOR = 0.2
GA_POP_SIZE = 10
GA_GEN = 10
MUT_RATE = 0.2

# Create grid
grid_res = 1
x_coords = np.arange(0, area_width, grid_res)
y_coords = np.arange(0, area_height, grid_res)
xx, yy = np.meshgrid(x_coords, y_coords)
GRID = np.c_[xx.ravel(), yy.ravel()]
dim = sensor_count * 3

# --- Fitness function aligned with hard constraint (Eq. 2)
def fitness(vec):
    total_cost = 0.0
    cover_cnt = np.zeros(len(GRID), dtype=int)
    for i in range(sensor_count):
        x, y, r = vec[i*3:i*3+3]
        dist = np.hypot(GRID[:, 0] - x, GRID[:, 1] - y)
        covered = dist <= r
        cover_cnt += covered.astype(int)
        total_cost += fixed_cost + COST_PER_RADIUS * r
    F1 = total_cost
    F2 = np.sum(cover_cnt >= MIN_COVERAGE_REQ)
    
    # Enforce hard constraint (if any point is uncovered, reject)
    if np.any(cover_cnt < MIN_COVERAGE_REQ):
        return float('inf'), F1, F2, cover_cnt

    return F1 / (F2 + EPS), F1, F2, cover_cnt

# Initialization
lb = np.tile([0, 0, r_min], sensor_count)
ub = np.tile([area_width, area_height, r_max], sensor_count)
v_max = (ub - lb) * V_MAX_FACTOR

pos = np.random.uniform(lb, ub, (SWARM_SIZE, dim))
vel = np.random.uniform(-v_max, v_max, (SWARM_SIZE, dim))
pbest_pos = pos.copy()
pbest_val = np.full(SWARM_SIZE, np.inf)
gbest_pos = None
gbest_val = np.inf
gbest_cov = None

# PSO main loop
for it in range(MAX_ITER):
    for i in range(SWARM_SIZE):
        f_val, _, _, heat = fitness(pos[i])
        if not np.isfinite(f_val):
            continue  # infeasible
        if f_val < pbest_val[i]:
            pbest_val[i] = f_val
            pbest_pos[i] = pos[i].copy()
        if f_val < gbest_val:
            gbest_val, gbest_pos, gbest_cov = f_val, pos[i].copy(), heat.copy()

    if gbest_pos is None:
        print(f"Iteration {it+1}: No feasible solution found.")
        continue

    # GA refinement around gbest
    pop = np.array([gbest_pos + np.random.normal(0, 0.05, dim) * (ub - lb)
                    for _ in range(GA_POP_SIZE)])
    pop = np.clip(pop, lb, ub)

    for _ in range(GA_GEN):
        fitnesses = [fitness(ind)[0] for ind in pop]
        best_idx = np.argsort(fitnesses)[:2]
        p1, p2 = pop[best_idx]
        crossover = np.where(np.random.rand(dim) < 0.5, p1, p2)
        if np.random.rand() < MUT_RATE:
            idx = np.random.randint(0, dim)
            crossover[idx] += np.random.normal(0, 0.1) * (ub[idx] - lb[idx])
        crossover = np.clip(crossover, lb, ub)
        c_fit = fitness(crossover)[0]
        if np.isfinite(c_fit) and c_fit < gbest_val:
            gbest_val, gbest_pos = c_fit, crossover.copy()

    # PSO velocity and position update
    r1, r2 = np.random.rand(SWARM_SIZE, dim), np.random.rand(SWARM_SIZE, dim)
    vel = (W_INERTIA * vel +
           C_COGNITIVE * r1 * (pbest_pos - pos) +
           C_SOCIAL * r2 * (gbest_pos - pos))
    vel = np.clip(vel, -v_max, v_max)
    pos = np.clip(pos + vel, lb, ub)

# Final evaluation
if gbest_pos is not None:
    final_fitness, final_cost, final_coverage, _ = fitness(gbest_pos)
    print("\n--- Final Optimization Result ---")
    print(f"Best Fitness (F1/F2)   : {final_fitness:.4f}")
    print(f"Total Cost (F1)        : {final_cost:.2f}")
    print(f"Covered Points (F2)    : {final_coverage}")
else:
    print("\n No feasible solution found after all iterations.")


Iteration 1: No feasible solution found.
Iteration 2: No feasible solution found.
Iteration 3: No feasible solution found.
Iteration 4: No feasible solution found.
Iteration 5: No feasible solution found.
Iteration 6: No feasible solution found.
Iteration 7: No feasible solution found.
Iteration 8: No feasible solution found.
Iteration 9: No feasible solution found.
Iteration 10: No feasible solution found.
Iteration 11: No feasible solution found.
Iteration 12: No feasible solution found.
Iteration 13: No feasible solution found.
Iteration 14: No feasible solution found.
Iteration 15: No feasible solution found.
Iteration 16: No feasible solution found.
Iteration 17: No feasible solution found.
Iteration 18: No feasible solution found.
Iteration 19: No feasible solution found.
Iteration 20: No feasible solution found.

 No feasible solution found after all iterations.
